# Análisis abonos casos parcializados $200.000

In [ ]:
import os
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

username = os.getlogin()
hoy = datetime.datetime.now()
dt_fmt = "%d.%m.%Y"
dt_fmtexport = "%d-%m-%Y"
dt_fmtcartola = "%d/%m/%Y"
dt_fmtcarga = "%Y%m%d"
dt_fmtcasos = "%d-%m-%Y %H:%M"
fecha_actual = hoy.strftime(dt_fmtexport)

lowmemory=False

df_casos = pd.DataFrame()

print("Cargando archivos...")
file_path = f'C:\\Users\\{username}\\Documents\\Py\\CASOS\\tablacasosanalisisparc.xlsx'
df_casos = pd.read_excel(file_path,
                    sheet_name='Historial casos Manuales',header=0,
                    names=['caso','agencia','tdep','contacto','asunto','cierre','montocierre','depositado','banco','user','fapertura','fmod','alias','usermod'])

folder_path = f'C:\\Users\\{username}\\Documents\\Py\\CARGASSAP\\ANALISIS\\CARGAS'

df_list = []
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',header=None,
                        engine='python',
                        #dtype='int64'
                        )
    df_list.append(df)
df_cargas = pd.concat(df_list)
df_cargas = df_cargas.dropna(axis=1)
df_cargas.columns = ['flowid','counter','account','period','post','constant1','sa','gl','monto','indcont','user','ref','agencia','doc']

df_casos['tdep'] = df_casos['tdep'].astype(str)
df_casos.fillna('null', inplace=True)
df_casos['fapertura'] = pd.to_datetime(df_casos['fapertura'],format = dt_fmtcasos)
df_casos['fmod'] = pd.to_datetime(df_casos['fmod'],format = dt_fmtexport)

df_cargas['agencia'] = df_cargas['agencia'].replace('AG. ','',regex=True)
df_cargas['agencia'] = df_cargas['agencia'].str.slice(0,-3)
df_cargas['post'] = pd.to_datetime((df_cargas['post'].astype(str)),format=dt_fmtcarga)
df_cargas['doc'] = pd.to_datetime((df_cargas['doc'].astype(str)),format=dt_fmtcarga)

print("Tablas cargadas.")

In [22]:
df_casos = df_casos[df_casos['fapertura'] > pd.to_datetime('10-01-2025',format=dt_fmtexport)]
df_casos = df_casos[df_casos['asunto'] != 'Conciliacion manual B2B - TS']

df_cargas_13 = df_cargas[df_cargas['doc'] == pd.to_datetime('13-01-2025',format=dt_fmtexport)]
df_cargas_14 = df_cargas[df_cargas['doc'] == pd.to_datetime('14-01-2025',format=dt_fmtexport)]

pd.to_numeric(df_cargas_13['agencia'],errors='coerce')
pd.to_numeric(df_cargas_14['agencia'],errors='coerce')
pd.to_numeric(df_casos['agencia'],errors='coerce')

#df_cargas['agencia'] = df_cargas['agencia'].astype(int)

df_cargas_adosc_13 = df_cargas_13[df_cargas_13['monto'] == 200000]
count_dosc_13 = df_cargas_adosc_13.shape[0]
count_adosc_13 = df_cargas_adosc_13.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)
count_adosc_13['agencia'] = count_adosc_13['agencia'].astype(int)

df_cargas_adosc_14 = df_cargas_14[df_cargas_14['monto'] == 200000]
count_dosc_14 = df_cargas_adosc_14.shape[0]
count_adosc_14 = df_cargas_adosc_14.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)
count_adosc_14['agencia'] = count_adosc_14['agencia'].astype(int)

df_casos_parc = df_casos[df_casos['tdep'] == "Parcializado"]
df_casos_pc = df_casos[df_casos['usermod']== "procom"]
df_casos_parc_pc = df_casos[(df_casos['tdep'] == "Parcializado") & (df_casos['usermod']== "procom")]


In [23]:
df_merge_13 = pd.merge(df_casos_parc,count_adosc_13, left_on='agencia', right_on='agencia')
df_merge_13_14 = pd.merge(count_adosc_13,count_adosc_14,on='agencia', how='outer')
df_countab = pd.concat([count_adosc_13,count_adosc_14],axis=1)

In [ ]:
nagencia = 223001
df_cargas_adosc_ag_13 = df_cargas_adosc_13[df_cargas_adosc_13['agencia'].astype(int) == nagencia]
df_cargas_adosc_ag_14 = df_cargas_adosc_14[df_cargas_adosc_14['agencia'].astype(int) == nagencia]

print (df_cargas_adosc_ag_13.shape[0])
print (df_cargas_adosc_ag_14.shape[0])
